In [1]:
#what happens if we build a circuit variational template but with basis gates that are parameters
#I think the guess, and eval functions wil be messed up but we can start with that
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

import sys
sys.path.append("../../")

# %matplotlib widget

In [21]:
from ast import Param
import src.utils.custom_gates as gates
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library.standard_gates import iSwapGate, CPhaseGate
a = Parameter('a')
gates.RiSwapGate(a)
#iSwapGate().power(1/a)
#CPhaseGate(a)

TypeError: ParameterExpression with unbound parameters ({Parameter(a)}) cannot be cast to an int.